In [ ]:
from intelhex import IntelHex
ih = IntelHex()
hexFileName = "2bitTT.hex"

In [ ]:
#---AND Truth table
#--- 0 - 1 -------
#--0 0 - 0 -------
#--1 0 - 1 -------
#To Write another way;
# 00:0
# 01:0
# 10:0
# 11:1
bit2Truth ={ 0:0, 1:0, 2:0, 3:1};
bit2Truth[3]
ih.fromdict(bit2Truth)


In [ ]:
from intelhex import IntelHex
class HexTruth():
    def __init__(csvFilename, hexFilename=None ):
        if hexFilename is None: hexFilename = csvFileName
        self.csvFilename
        self.ih= IntelHex()

In [ ]:
16*28 

In [ ]:
ih.addresses()

In [1]:
def writeFromIntelHex(ih, fileName="test.hex"):
    f = open(fileName, 'w');
    ih.write_hex_file(f)
    f.close();

In [2]:
csvFileName = 'b2TruthTable.csv'
def importCSV(csvFileName):
    import csv
    csvTruth = []
    with open(csvFileName, newline='') as csvfile:
        csvReadingMachine = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in csvReadingMachine:
            csvTruth.append(row)
    return csvTruth

def parseCSV(csvTruth, rangeIn = [0,8], rangeOut = [10]):
    hexDict = {};
    for line in csvTruth:
        binaryString = ""
        for j in rangeIn:
            if "#" in str(j):break
            binaryString+=str(line[j])
        address = (int(binaryString,2))
        
        binaryString = ""
        for j in rangeOut:
            if "#" in str(j):break
            binaryString+=str(line[j])
        data = (int(binaryString,2))
        hexDict[address] = data
    return hexDict

        

In [4]:
from intelhex import IntelHex
ih = IntelHex()
csvList = importCSV("b2TruthTable.csv")
hexDict = parseCSV(csvList, rangeIn = [0,1], rangeOut = [3])
ih.fromdict(hexDict)
ih.dump()
writeFromIntelHex(ih)

0000  00 00 00 01 -- -- -- -- -- -- -- -- -- -- -- --  |....            |


/home/jake/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  app.launch_new_instance()


In [11]:
ih.dump()


0000  00 00 00 01 -- -- -- -- -- -- -- -- -- -- -- --  |....            |


In [ ]:
synthesizeHex()

In [ ]:
hex(110)

In [ ]:
ih.dump()


In [ ]:
hex(int(("0110"),2))